## Computing time

We compare the computing time of our method with those of other existing methods, including Hopfield approximation (HF), Maximum Likelihood Estimation (MLE), and Pseudo Likelihood Estimations (PLE). The code for PLE was adapted from ConIII package (https://github.com/eltrompetero/coniii).

As shown in the previous sections, MLE cannot work for a large system such as 40 variables and PLE cannot work for a very large system size such as 100 variables. In the following, we will consider a small system with 20 variables. The computing time of MLE and PLE increases rapidly as system size increases (see our paper for more detail).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import emachine as EM
import timeit

In [2]:
np.random.seed(0)

We set parameters.

In [3]:
n_var = 20
g = 1.0
n_seq = 400

Variable configurations are generated.

In [4]:
w_true,seqs = EM.generate_seq(n_var,n_seq,g=g)

In [5]:
ops = EM.operators(seqs)

In [6]:
# Epsilon Machine (EM)
start_time = timeit.default_timer()

eps_list = np.linspace(0.5,0.9,9)
E_eps = np.zeros(len(eps_list))
w_eps = np.zeros((len(eps_list),ops.shape[1]))
for i,eps in enumerate(eps_list):    
    w_eps[i,:],E_eps[i] = EM.fit(ops,eps=eps,max_iter=100)
    #print(eps,E_eps[i])    
ieps = np.argmax(E_eps)
#print('The optimal value of eps:',eps_list[ieps])
w_em = w_eps[ieps]

run_time_em = timeit.default_timer() - start_time

In [7]:
# Hopfield approximation (HF)
start_time = timeit.default_timer()
w_hf = EM.hopfield_method(seqs)
run_time_hf = timeit.default_timer() - start_time

In [8]:
# Maximum Likelihood Estimation (MLE)
start_time = timeit.default_timer()
w_mle = EM.MLE_method(seqs)
run_time_mle = timeit.default_timer() - start_time

In [9]:
# Pseudo Likelihood Estimation (PLE)
start_time = timeit.default_timer()
w_ple = EM.PLE_method(seqs)
run_time_ple = timeit.default_timer() - start_time

We compare the performance and computing time of these methods.

In [11]:
MSE_em = ((w_true - w_em)**2).mean()
MSE_hf = ((w_true - w_hf)**2).mean()
MSE_mle = ((w_true - w_mle)**2).mean()
MSE_ple = ((w_true - w_ple)**2).mean()

df = pd.DataFrame([['EM',MSE_em,run_time_em],
                   ['HF',MSE_hf,run_time_hf],
                   ['MLE',MSE_mle,run_time_mle],
                   ['PLE',MSE_ple,run_time_ple]],
                  columns = ['Method','Mean squared error','Computing time (s)'])
df

,Method,Mean squared error,Computing time (s)
0,EM,0.017399,0.282543
1,HF,0.087274,0.001733
2,MLE,0.025202,290.850171
3,PLE,0.052717,2.742374
